# Biblioteca de generación y manejo de grafos

Biblioteca orientada a objetos, para describir y utilizar grafos. 

In [1]:
import random
import numpy as np
import math as mt
import matplotlib.pyplot as plt
import copy
import time as tm
import os
import imageio

# Clases

## Nodo

In [2]:
class Node:
  def __init__(self, name, x = 0, y = 0, deg = 0):
    # propiedades
    self.name = name
    self.x_coord = x
    self.y_cord = y
    self.degree = deg
    self.weight = 0
  
  # funciones  
  def get_name(self):
    return self.name
  def get_degree(self):
    return self.degree
  def add_degree(self):
    self.degree += 1

## Arista

In [3]:
class Edge:
  def __init__(self, start, end, d = 0, w = 0):
    # propiedades
    self.start = start
    self.end = end
    self.dist = d
    self.weight = w
  
  # funciones
  def edge_start(self):
    return self.start
  def edge_end(self):
    return self.end
  def edge_nodes(self):
    return [self.start, self.end]

## Grafo

In [4]:
class Graph:
  def __init__(self, name, gtype = "graph", path = False):
    # propiedades
    self.name = name
    self.nodes = []
    self.edges = []
    self.gtype = gtype
    # cuando hay caminos
    self.path = path
    self.node_list = []
    self.edge_list = []
    self.weight_list = []

  # acciones nodos
  def add_nodes(self, Lnodes):
    self.nodes.extend(Lnodes)
  def get_nodes(self):
    return self.nodes
  # acciones aristas
  def add_edges(self, Ledges):
    self.edges.extend(Ledges)
  def get_edges(self):
    return self.edges

############################# Función de exportación #####################################

  def save_graph(self, filename, rand_weight = False):
    infile = self.gtype + " " + self.name + " {\n"
    
    # no dirigido
    if self.gtype == "graph":
      edge_type = " -- "
    # dirigido  
    else:
      edge_type = " -> "
    
    # en caso de camino
    if self.path:
      for i in range(len(self.nodes)):
        if i in self.node_list:
          node_position = self.node_list.index(i)
          infile += str(self.nodes[i].name) + " [label=nd_" + str(self.nodes[i].name) + "(" + str(self.weight_list[node_position]) + ")];\n"
        else: 
          infile += str(self.nodes[i].name) + ";\n"

      for i in range(len(self.edge_list)):
        infile += str(self.edge_list[i][0]) + edge_type + str(self.edge_list[i][1]) + " [color = pink,] ;\n"
    # sin camino
    else:
      for i in range(len(self.nodes)):
        infile += str(self.nodes[i].name) + ";\n"
    
    # aristas con peso/distancia
    if rand_weight:
      for i in range(len(self.edges)):
        infile += str(self.edges[i].start) + edge_type + str(self.edges[i].end) + "[weight=\"" + str(round(random.random() * 4) + 1) + "\"]" + ";\n"
    elif self.path:
      for i in range(len(self.edges)):
        infile += str(self.edges[i].start) + edge_type + str(self.edges[i].end) + "[weight=\"" + str(self.edge[i].weight) + "\"]" + ";\n"            
    else:
      for i in range(len(self.edges)):
        infile += str(self.edges[i].start) + edge_type + str(self.edges[i].end) + " [color=black];\n"
      
    infile += "}"

    # genera archivo GraphViz    
    with open(filename + '.gv', 'w') as outfile:
      outfile.write(infile)
    
    return infile

# 1. Modelos de generación de grafos

### Malla

In [5]:
# malla de n filas * m columnas
def Grid(n, m, name, direc = False):
  # inicial  
  nodes = []
  edges = []    

  node_count = 0
  ########### genera nodos ###########
  for i in range(n):
    for j in range(m):    
      nodes.append(Node(node_count, x = j, y = i))
      node_count += 1

  ########### genera aristas ###########
  for j in range(m-1):
    # orilla derecha
    edges.append(Edge(n-1+i*n,2*n-1+i*n))
    for i in range(n-1):
      # orilla superior
      edges.append(Edge(n*(m-1) + i, n*(m-1) + i + 1))
      # aristas generales
      edges.append(Edge(i + n*j, i + n*j + 1))
      edges.append(Edge(i + n*j, i+ n*j + n))
  
  '''
  #Hasta la derecha
  for i in range(m-1): #n es el ancho, m es la altura
    Aris.append(Arista(n-1+i*n,2*n-1+i*n)) 
  #Hasta arriba
  for i in range(n-1): #n es el ancho, m es la altura
    Aris.append(Arista(n*(m-1)+i,n*(m-1)+i+1))
  
  if pacman: #si es un mundo pacman y el final es el inicio :o Ö o.o O_O
    #Conectamos derecha con izquierda
    for i in range(m):
      Aris.append(Arista(n-1+i*n,i*n))
    #Conectamos arriba con abajo
    for i in range(n):
      Aris.append(Arista(n*(m-1)+i,i))
  '''
  # dirigido    
  if direc:
    new_graph = Graph(name, gtype = "digraph")
  # no dirigido 
  else:
    new_graph = Graph(name)
  
  # G(N,E)
  new_graph.add_nodes(nodes)
  new_graph.add_edges(edges)
  
  return new_graph

### Erdös-Renyi

In [6]:
def Erdos_Renyi(n, m, name, direc = False, auto = False):
  # inicial
  adj_matrix = np.zeros((n, n), dtype = int) 
  nodes = []
  edges = []    
  
  ########### genera nodos ###########
  for i in range(n):
    nodes.append(Node(i))
  
  ########### genera aristas ###########
  for i in range(m):  
    while True:    
      start = random.randint(0, n-1)
      end = random.randint(0, n-1)
      # asegurando nodos diferentes
      if ( (start != end) or auto ) and adj_matrix[start][end] == 0:
        break
    adj_matrix[start][end] = 1
    # guarda arista nueva
    edges.append(Edge(start, end))
    # espejo para grafo no dirigido
    if not direc:
      adj_matrix[end][start] = 1

  # dirigido    
  if direc:
    new_graph = Graph(name, gtype = "digraph")
  # no dirigido 
  else:
    new_graph = Graph(name)
  
  # G(N,E)
  new_graph.add_nodes(nodes)
  new_graph.add_edges(edges)
  
  return new_graph

### Gilbert

In [7]:
def Gilbert(n, p, name, direc = False, auto = False): 
  # inicial
  nodes = []
  edges = []  
  
  ########### genera nodos ###########  
  for i in range(n):
    nodes.append(Node(i))

  ########### genera aristas ########### 
  edge_lim = n + auto - 1

  for i in range(n):
    if not direc:
      lim = i + auto
    for j in range(lim):
      # condición probabilística (Gilbert)
      if random.random() <= p:
        if auto or (not i == j):
          edges.append(Edge(i, j))

  # dirigido    
  if direc:
    new_graph = Graph(name, gtype = "digraph")
  # no dirigido 
  else:
    new_graph = Graph(name)
  
  # G(N,E)
  new_graph.add_nodes(nodes)
  new_graph.add_edges(edges)
  
  return new_graph

### Geográfico simple

In [8]:
def eucl_dist(x1, y1, x2, y2):
  dist = mt.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
  return dist

In [9]:
def Simple_Geo(n, r, name, direc = False, auto = False):    
  # inicial
  nodes = []
  edges = [] 

  x_coord = []
  y_coord = []
  
  ########### genera nodos ###########
  for i in range (n):
    x = random.random()
    x_coord.append(x)

    y = random.random()
    y_coord.append(y)
    
    nodes.append(Node(i,x = x, y = y))
  
  '''
  # gráfica
  for i in range(n):  
    plt.text(x_coord[i], y_coord[i], i)
  '''

  ########### genera aristas ###########  
  for i in range(n):
    for j in range(n - i + auto - 1):
      dist = eucl_dist(x_coord[i], y_coord[i], x_coord[n-j-1], y_coord[n-j-1])
      # condición de distancia
      if dist < r:
        edges.append(Edge(i, n-j-1, d = dist))
        #plt.plot([x_coord[i], x_coord[n-j-1]], [y_coord[i], y_coord[n-j-1]])
  
  # dirigido    
  if direc:
    new_graph = Graph(name, gtype = "digraph")
  # no dirigido 
  else:
    new_graph = Graph(name)
  
  # G(N,E)
  new_graph.add_nodes(nodes)
  new_graph.add_edges(edges)
  
  return new_graph

### Barabasi-Albert

In [10]:
def Barabasi_Albert(n, d, name, direc = False, auto = False):
  # inicial
  nodes = []
  edges = [] 

  n_coord = []
  
  ########### genera nodos ###########    
  for i in range(n): #generamos nuevo nodo
    nodes.append(Node(i))
    n_coord.append(0)
    ########### genera aristas ###########
    for j in range (0, i): # comparación con nodos existentes
      p = 1 - n_coord[j] / d
      if random.random() < p:
        edges.append(Edge(i, j))

        n_coord[i] += 1
        n_coord[j] += 1
        nodes[i].add_degree()
        nodes[j].add_degree()

  # dirigido    
  if direc:
    new_graph = Graph(name, gtype = "digraph")
  # no dirigido 
  else:
    new_graph = Graph(name)
  
  # G(N,E)
  new_graph.add_nodes(nodes)
  new_graph.add_edges(edges)
  
  return new_graph 

### Dorogovtsev-Mendes

In [11]:
def Dorogovtsev_Mendes(n, name, direc = False, limit = 0):
  # inicial
  nodes = []
  edges = [] 
  
  # Primeros tres nodos
  for i in range(3):
    nodes.append(Node(i))

  edges.append(Edge(0,1))
  edges.append(Edge(1,2))
  edges.append(Edge(2,0))

  if limit:
    for i in range(3):
      nodes[i].add_degree()
      nodes[i].add_degree()
    #nodes[0].add_degree()
    #nodes[1].add_degree()
    #nodes[2].add_degree()
    #nodes[0].add_degree()
    #nodes[1].add_degree()
    #nodes[2].add_degree()

  ########### genera nodos ###########   
  for i in range(n - 3):
    nodes.append(Node(i + 3))

    if limit:
      flagged = True
      while flagged:
        pick = random.randint(0, len(edges) - 1)
        if nodes[edges[pick].edge_start()].get_degree() < limit and nodes[edges[pick].edge_end()].get_degree() < limit:
          flagged = False

    ########### genera aristas ########### 
    else:
      pick = random.randint(0, len(edges)-1)
    edges.append(Edge(i+3, edges[pick].edge_start()))
    edges.append(Edge(i+3, edges[pick].edge_end()))

    if limit:
      nodes[-1].add_degree()
      nodes[-1].add_degree()
      nodes[edges[pick].edge_start()].add_degree()
      nodes[edges[pick].edge_end()].add_degree()

  # dirigido    
  if direc:
    new_graph = Graph(name, gtype = "digraph")
  # no dirigido 
  else:
    new_graph = Graph(name)
  
  # G(N,E)
  new_graph.add_nodes(nodes)
  new_graph.add_edges(edges)
  
  return new_graph

## Archivos p1

In [12]:
generated_graphs = []
# Malla
generated_graphs.append(Grid(3, 10, "Malla_30"))
generated_graphs.append(Grid(10, 10, "Malla_100"))
generated_graphs.append(Grid(20, 25, "Malla_500"))
#Erdös-Renyi
generated_graphs.append(Erdos_Renyi(30, 50, "E-R_30_m50"))
generated_graphs.append(Erdos_Renyi(100, 200, "E-R_100_m200"))
generated_graphs.append(Erdos_Renyi(500, 1000, "E-R_500_m1000"))
#Gilbert
generated_graphs.append(Gilbert(30,.5,"Gilbert_30_p50"))
generated_graphs.append(Gilbert(100,.3,"Gilbert_100_p30"))
generated_graphs.append(Gilbert(500,.2,"Gilbert_500_p20"))
# geográfico simple
generated_graphs.append(Simple_Geo(30,.2,"Geo_30_r2"))
generated_graphs.append(Simple_Geo(100,.4,"Geo_100_r4"))
generated_graphs.append(Simple_Geo(500,.7,"Geo_500_r7"))
# BarabasiAlbert
generated_graphs.append(Barabasi_Albert(30,5,"B-A_30_5"))
generated_graphs.append(Barabasi_Albert(100,3,"B-A_100_3"))
generated_graphs.append(Barabasi_Albert(500,10,"B-A_500_10"))
#Dorogovtsev Mendes
generated_graphs.append(Dorogovtsev_Mendes(30,"D-Mendes_30"))
generated_graphs.append(Dorogovtsev_Mendes(100,"D-Mendes_100"))
generated_graphs.append(Dorogovtsev_Mendes(500,"D-Mendes_500"))

In [13]:
for i in range(len(generated_graphs)):
    generated_graphs[i].save_graph(generated_graphs[i].name)

# 2. Algoritmos de árboles (BFS y DFS) 

## BFS

In [14]:
def BFS(graph, s): #Búsqueda a lo Ancho, s= nodo fuente
  og_graph = copy.deepcopy(graph)
  # inicial
  nodes = []
  edges = []

  nodes.append(Node(s))
  
  lst = [s]
  switch = 1
  added = 1

  while switch:
    switch=0
    #rng=range(added)
    added = 0
    new_node_lst=[]

    for i in range(added):
      remove = []
     
      for j in range(len(og_graph.edges)): 
        # Si existe la arista
        if (lst[- i - 1] in og_graph.edges[j].edge_nodes()): 
          # nodos
          if og_graph.edges[j].edge_nodes().index(lst[- i - 1]): 
            new_node = og_graph.edges[j].edge_nodes()[0]

          else:
            new_node = og_graph.edges[j].edge_nodes()[1]  

          if (new_node not in lst) and (new_node not in new_node_lst): 
            new_node_lst.append(new_node) 
            # aristas
            edges.append(og_graph.edges[j])

            added += 1
            switch = 1
          
          remove.append(j) # checada

      for k in range(len(remove)):
        og_graph.edges.pop(remove[- k - 1])
    
    for k in range(len(new_node_lst)):    
      nodes.append(Node(new_node_lst[k]))
    lst.extend(new_node_lst)

  name = og_graph.name + "_BFS"

  # G(N,E)
  new_graph= Graph(name)
  new_graph.add_nodes(nodes)
  new_graph.add_edges(edges)

  return new_graph

## DFS

### DFS-R

In [15]:
def DFS_R(graph, s):
  og_graph = copy.deepcopy(graph)
  # inicial
  nodes = []
  edges = []

  lst = [s]
  def dig(node, lst, og_graph, edges):    
    for j in range(len(og_graph.edges)): # todas 
      if node in og_graph.edges[j].edge_nodes(): # arista existente
        if og_graph.edges[j].edge_nodes().index(node): 
          # Nuevo nodo 
          new_node = og_graph.edges[j].edge_nodes()[0]
        else:
          new_node = og_graph.edges[j].edge_nodes()[1]
        if new_node not in lst: 
          # nueva arista
          edges.append(Edge(node, new_node))
          lst.append(new_node)
          lst,edges = dig(new_node, lst, og_graph, edges) 
    return lst, edges

  # nodos y aristas
  node_lst,edges = dig(s, lst, og_graph, edges)
  for i in range(len(node_lst)):
    nodes.append(Node(lst[i]))
  
  # G(N,E)
  name = og_graph.name + "_DFS-R"
  new_graph= Graph(name)
  new_graph.add_nodes(nodes)
  new_graph.add_edges(edges)

  return new_graph

### DFS-I

In [16]:
def DFS_I(graph, s):
  og_graph = copy.deepcopy(graph)
  # inicial
  nodes = []
  edges = []

  nodes.append(Node(s))
  lst = [s]
  
  pos=0
  while pos<len(lst):
    node=lst[pos]
    for j in range(len(og_graph.edges)): #Para todas las aristas
      if node in og_graph.edges[j].edge_nodes(): #Si está
        if og_graph.edges[j].edge_nodes().index(node): #Si está en la 2da posición
          new_node=og_graph.edges[j].edge_nodes()[0]
        else:
          new_node=og_graph.edges[j].edge_nodes()[1]
        if new_node not in lst: #el otro valor no en L, nos vamos hacia allá
          lst=[new_node]+lst
          edges.append(Edge(node,new_node))
          nodes.append(Node(new_node))
          suma=0
          pos=0
          break
    pos+=suma
    suma=1
  
  # G(N,E)
  name = og_graph.name + "_DFS-I"
  new_graph= Graph(name)
  new_graph.add_nodes(nodes)
  new_graph.add_edges(edges)

  return new_graph

## Archivos (P2)

In [17]:
for i in range(len(generated_graphs)):
  a = BFS(generated_graphs[i], random.randint(0, len(generated_graphs[i].nodes) - 1)) 
  a.save_graph(a.name)

  b = DFS_R(generated_graphs[i], random.randint(0, len(generated_graphs[i].nodes) - 1))
  b.save_graph(b.name)

  c = DFS_I(generated_graphs[i], random.randint(0, len(generated_graphs[i].nodes) - 1))
  c.save_graph(c.name)

# 3. Algoritmo de Dijkstra

In [18]:
def Dijkstra(graph):
  new_graph = copy.deepcopy(graph)

  new_node = len(new_graph.nodes)
  start = round(random.random() * (new_node - 1))
  
  ## Árbol de mínima expansión ##
 
  n_matrix = np.zeros((new_node, new_node), dtype = int)
  
  for i in range(len(new_graph.edges)):
    n_matrix[new_graph.edges[i].start][new_graph.edges[i].end] = new_graph.edges[i].weight
    n_matrix[new_graph.edges[i].end][new_graph.edges[i].start] = new_graph.edges[i].weight
  
  # inicial
  known_nodes = [start] 
  known_nodes_w = [0] # pesos
  known_edges = [] 
  # adyacentes
  neighbor_nodes = np.asarray(np.where(n_matrix[start] > 0)[0])     
  neighbor_edges = []  
  
  for i in range(len(neighbor_nodes)):
    neighbor_edges.append([start,neighbor_nodes[i]])
  # pesos 
  neighbor_edges_w = n_matrix[start][neighbor_nodes]   
  neighbor_nodes_w = n_matrix[start][neighbor_nodes]   
    
  # regresa los recorridos a peso 0
  for i in range(new_node):
    n_matrix[start][i] = 0
    n_matrix[i][start] = 0    

  while len(neighbor_nodes):
    pick = np.where(node_lst_w==min(node_lst_w))[0] # escoge peso min
    pick = pick[0]
    # en camino
    known_edges.append(neighbor_edges[pick])
    known_nodes.append(neighbor_nodes[pick])
    known_nodes_w.append(neighbor_nodes_w[pick])
        
    # generados
    node_pick = neighbor_nodes[pick]
    new_node_lst = np.where(n_matrix[node_pick] > 0)[0]
    new_edge_lst=[]
    
    for i in range(len(new_node_lst)):
      new_edge_lst.append([node_pick, new_node_lst[i]])
    
    new_edge_lst_w = n_matrix[node_pick][new_node_lst]
    new_node_lst_w = new_edge_lst_w + node_lst_w[pick]

    # update para nodos nuevos   
    for i in range(len(new_node_lst)):
      if new_node_lst[i] not in neighbor_nodes: 
        neighbor_nodes = np.append(neighbor_nodes, new_node_lst[i])
        neighbor_edges.append(new_edge_lst[i])
        edge_lst_w = np.append(edge_lst_w, new_edge_lst_w[i])
        node_lst_w = np.append(node_lst_w, new_node_lst_w[i])
      
      #existe camino al nodo
      else:       
        maybe = np.where(neighbor_nodes == new_node_lst[i])[0] # posición 
        maybe = maybe[0]
        # peso min 
        if node_lst_w[maybe] > new_node_lst_w[i]: 
          neighbor_edges[maybe] = new_edge_lst[i]
          node_lst_w[maybe] = new_node_lst_w[i]
          edge_lst_w[maybe] = new_edge_lst_w[i]
   
    # evita repetición
    for i in range(new_node):
      n_matrix[node_pick][i]=0
      n_matrix[i][node_pick]=0

    # update adyacentes
    neighbor_nodes = np.delete(neighbor_nodes, pick)
    neighbor_edges.pop(pick)
    node_lst_w = np.delete(node_lst_w,pick)
    edge_lst_w = np.delete(edge_lst_w,pick)    

  ### camino
  path = []
  for i in range(len(known_edges)):
    if known_edges[i][1] == end:
      path.append(known_edges[i])    
  while path[-1][0] != start:
    for i in range(len(known_edges)):
      if known_edges[i][1]==path[-1][0]:
        path.append(known_edges[i])    
               
  # G(E,V)
  new_graph.name = graph.name + "_" + str(start)
  new_graph.path = True
  new_graph.edge_list = known_edges
  new_graph.node_list = known_nodes
  new_graph.weight_list = known_nodes_w
  
  return new_graph


## Archivos (P3)

In [19]:
def rand_weight(graph, max_w):    
  new_graph = copy.deepcopy(graph)
  for i in range(len(new_graph.edges)):
    new_graph.edges[i].weight = int(round(random.random() * (max_w - 1)) + 1)
  
  return new_graph

In [20]:
generated_graphs = []
# Malla
generated_graphs.append(Grid(10, 10, "Malla_100"))
generated_graphs.append(Grid(200, 25, "Malla_5000"))
#Erdös-Renyi
generated_graphs.append(Erdos_Renyi(100, 200, "E-R_100_m200"))
generated_graphs.append(Erdos_Renyi(5000, 1000, "E-R_5000_m1000"))
#Gilbert
generated_graphs.append(Gilbert(100,.3,"Gilbert_100_p30"))
generated_graphs.append(Gilbert(5000,.2,"Gilbert_5000_p20"))
# geográfico simple
generated_graphs.append(Simple_Geo(100,.4,"Geo_100_r4"))
generated_graphs.append(Simple_Geo(5000,.7,"Geo_5000_r7"))
# BarabasiAlbert
generated_graphs.append(Barabasi_Albert(100,3,"B-A_100_3"))
generated_graphs.append(Barabasi_Albert(5000,10,"B-A_500_10"))
#Dorogovtsev Mendes
generated_graphs.append(Dorogovtsev_Mendes(100,"D-Mendes_100"))
generated_graphs.append(Dorogovtsev_Mendes(5000,"D-Mendes_5000"))

In [21]:
for i in range(len(generated_graphs)):
    generated_graphs[i] = rand_weight(generated_graphs[i], 100)
    generated_graphs[i].save_graph(generated_graphs[i].name)
    a = Dijkstra(generated_graphs[i])
    a.save_graph(a.name)

# 4. Kruskal y Prim

## Kruskal

In [22]:
def choose_weight(elem):
  return elem[2]  

### Kruskal directo

In [23]:
def Kruskal_D(graph):
  new_graph = copy.deepcopy(a)

  edges_lst = []
  for i in range(len(new_graph.edges)):
    edges_lst.append([new_graph.edges[i].start, new_graph.edges[i].end, new_graph.edges[i].weight])
  edges_lst.sort(key = choose_weight)
  
  # unión
  pairs = []
  edges_lst_n = []

  #Inicial
  pairs.append([edges_lst[0][0], edges_lst[0][1]]) #El primer conjunto
    
  edges_lst_n.append([edges_lst[0][0], edges_lst[0][1]])
  exp_w = edges_lst[0][2]
  edges_lst.pop(0)
  for i in range(len(new_graph.edges)):
    for j in range(len(edges_lst)):
      same = False
      p_pairs = False
      s_pairs = False
      p_index = 0
      s_index = 0

      for k in range(len(pairs)):        
        p_pairs = (edges_lst[j][0] in pairs[k]) or p_pairs
        if edges_lst[j][0] in pairs[k]:
          p_index = k
        if edges_lst[j][1] in pairs[k]:
          s_index = k
        s_pairs = (edges_lst[j][1] in pairs[k]) or s_pairs
        same = ((edges_lst[j][0] in pairs[k]) and (edges_lst[j][1] in pairs[k])) or same

      # pop en en mismo conjunto     
      if p_pairs and s_pairs and same: 
        edges_lst.pop(j)   
        break
      # union si cada una en un cjto  
      elif p_pairs and s_pairs and not same: 
        pairs[p_index].extend(pairs[s_index])   
        pairs.pop(s_index)

        edges_lst_n.append([edges_lst[j][0], edges_lst[j][1]])
        exp_w+=edges_lst[j][2]
        edges_lst.pop(j)
        break
      # 1 en cjto
      elif p_pairs and not s_pairs:   
        pairs[p_index].extend([edges_lst[j][1]])
        edges_lst_n.append([edges_lst[j][0],edges_lst[j][1]])
        exp_w+=edges_lst[j][2]
        edges_lst.pop(j)
        break
      # 2 en cjto
      elif not p_pairs and s_pairs:   
        pairs[p_index].extend([edges_lst[j][0]]) 
        edges_lst_n.append([edges_lst[j][0], edges_lst[j][1]])
        exp_w+=edges_lst[j][2]
        edges_lst.pop(j)
        break

  new_graph.name = new_graph.Nom + "_Kr-D_" + str(exp_w)
  new_graph.path = True
  new_graph.edge_list = edges_lst_n

  return new_graph


### Kruskal inverso

In [24]:
def Kruskal_I(graph):
  new_graph=copy.deepcopy(graph)
  edges = []
  
  for i in range(len(new_graph.edges)):
    edges.append([new_graph.edges[i].start,new_graph.edges[i].end,new_graph.edges[i].weight,i])
  edges.sort(reverse = True, key = choose_weight)
    
  mid_graph=copy.deepcopy(new_graph)
  aux = DFS_I(mid_graph, 0)
  node_con = len(aux.nodes)
  j = 0

  while len(edges) != (node_con-1) :
    temp_graph = copy.deepcopy(mid_graph)
    temp_graph.edges.pop(edges[j][3])
    aux = DFS_R(temp_graph,0)
    if len(aux.nodes) == node_con: 
      mid_graph = copy.deepcopy(temp_graph)
      #Actualizar aristas
      for k in range(len(edges)):
        if edges[k][3] > edges[j][3]:
          edges[k][3] = edges[k][3]-1
      edges.pop(j)
    else:
      j += 1
  exp_weight = 0
  edges_lst_n = []
  for i in range(len(edges)):
    exp_weight += edges[i][2]
    edges_lst_n.append([edges[i][0], edges[i][1]])
  
  new_graph.name = new_graph.name + "_KrI_" + str(exp_weight)
  new_graph.path = True
  new_graph.edge_list = edges_lst_n

  return new_graph

## Prim

In [25]:
def Prim(a):
  new_graph=copy.deepcopy(a)
  
  edge_lst=[]
  for i in range(len(new_graph.edges)):
    edge_lst.append([new_graph.edges[i].start,new_graph.edges[i].end,new_graph.edges[i].weight])
  edge_lst.sort(key = choose_weight)
    
  pairs=[]
  edge_lst_n=[]
  
  # inicial
  pairs.append(edge_lst[0][0])
  pairs.append(edge_lst[0][1])
  edge_lst_n.append([edge_lst[0][0],edge_lst[0][1]])
  exp_weight=edge_lst[0][2]
  edge_lst.pop(0)
    
  for i in range(len(new_graph.edges)):
    for j in range(len(edge_lst)):
        p_pairs = edge_lst[j][0] in pairs
        s_pairs = edge_lst[j][1] in pairs
        
        if p_pairs and s_pairs:
          edge_lst.pop(j)
          break
        
        elif p_pairs and not s_pairs:
          pairs.append(edge_lst[j][1])
          edge_lst_n.append([edge_lst[j][0],edge_lst[j][1]])
          exp_weight += edge_lst[j][2]
          edge_lst.pop(j)
          break

        elif not p_pairs and s_pairs:
          pairs.append(edge_lst[j][0])
          edge_lst_n.append([edge_lst[j][0], edge_lst[j][1]])
          exp_weight += edge_lst[j][2]
          edge_lst.pop(j)
          break

  new_graph.name = new_graph.name + "_Prim_" + str(exp_weight)  
  new_graph.path = True
  new_graph.edge_list = edge_lst_n
  return new_graph

## Archivos (P4)

In [28]:
for i in range(len(generated_graphs)):
  a = Kruskal_D(generated_graphs[i]) 
  a.save_graph(a.Name)

  b = Kruskal_I(generated_graphs[i])
  b.save_graph(b.Name)

  c = Prim(generated_graphs[i])
  c.save_graph(c.Name)